In [1]:
import polars as pl

from pathlib import Path
from multiprocessing import Pool
import tempfile
import subprocess
import tqdm

In [2]:
OUTPUT_PATH = Path('../data-raw/uv/')
TIME_DELTA = pl.duration(days=1)  # Simulate at release date + TIME_DELTA
WORKERS = 12

In [34]:
def simulate_installation(package, release, date, filename):
    with tempfile.TemporaryDirectory() as directory:
        # Initialize a uv project
        try:
            subprocess.run(['uv', 'init', '.', '--bare', '--no-workspace', '--name', 'root'], cwd=directory, check=True, capture_output=True, text=True)
        except subprocess.CalledProcessError as e:
            with open(OUTPUT_PATH / f'{filename}.err', 'w') as fp:
                fp.write(e.stdout)
                fp.write('\n\n\n')
                fp.write(e.stderr)
            raise
            
        # Add dependency
        try:
            r = subprocess.run(
                ['uv', 'add', f'{package}=={release}', '--exclude-newer', date, '--no-sync', '-qq', '--color', 'never'], 
                cwd=directory, check=True, capture_output=True, text=True
            )
        except subprocess.CalledProcessError as e:
            with open(OUTPUT_PATH / f'{filename}.err', 'w') as fp:
                fp.write(e.stdout)
                fp.write('\n\n\n')
                fp.write(e.stderr)
            raise

        # Export tree
        r = subprocess.run(
            ['uv', 'tree', '--no-dev', '--no-dedupe', '--no-progress', '--show-sizes', '--quiet', '--color', 'never'], 
            cwd=directory, capture_output=True, text=True
        )
        with open(OUTPUT_PATH / f'{filename}.tree', 'w') as fp:
            fp.write(r.stdout)

        """
        # Export lockfile
        r = subprocess.run(
            ['uv', 'export', '--frozen', '--format', 'pylock.toml'], 
            cwd=directory, capture_output=True, text=True
        )
        with open(OUTPUT_PATH / f'{filename}.lock', 'w') as fp:
            fp.write(r.stdout)
        """
            
        

In [26]:
packages = (
    pl.read_parquet('../data/selected_releases.parquet')
    .sort('package', 'date')
    .select(
        'package', 
        'release', 
        (pl.col('date') + TIME_DELTA).dt.to_string('%FT%T'),
    )
    .with_columns(pl.col('date').shift(-1).over('package').alias('date_next'))
    .rows()
)

In [33]:
for package, release, date, next_date in tqdm.tqdm(packages, miniters=1): 
    filename = f'{package}#{release}#current'
    if not Path(OUTPUT_PATH / (filename + '.tree')).exists():
        try:
            simulate_installation(package, release, date, filename)
        except Exception as e:
            print(filename, type(e), e)
            
    filename = f'{package}#{release}#next'
    if not Path(OUTPUT_PATH / (filename + '.tree')).exists() and next_date is not None:
        try:
            simulate_installation(package, release, next_date, filename)
        except Exception as e:
            print(filename, type(e), e)


  0%|                                       | 71/48380 [00:00<06:22, 126.22it/s]

CairoSVG#2.6.0#current <class 'subprocess.CalledProcessError'> Command '['uv', 'add', 'CairoSVG==2.6.0', '--exclude-newer', '2023-01-13T16:55:31', '--no-sync', '-qq', '--color', 'never']' returned non-zero exit status 1.


  1%|▍                                     | 478/48380 [00:01<01:34, 508.20it/s]

ImageIO#2.24.0#current <class 'subprocess.CalledProcessError'> Command '['uv', 'add', 'ImageIO==2.24.0', '--exclude-newer', '2023-01-10T02:51:12', '--no-sync', '-qq', '--color', 'never']' returned non-zero exit status 1.
PureCloudPlatformClientV2#161.0.0#current <class 'subprocess.CalledProcessError'> Command '['uv', 'add', 'PureCloudPlatformClientV2==161.0.0', '--exclude-newer', '2023-01-12T07:28:30', '--no-sync', '-qq', '--color', 'never']' returned non-zero exit status 1.


  1%|▍                                     | 600/48380 [00:02<02:51, 278.20it/s]

...

PyMuPDF#1.23.17#current <class 'subprocess.CalledProcessError'> Command '['uv', 'add', 'PyMuPDF==1.23.17', '--exclude-newer', '2024-01-24T00:47:28', '--no-sync', '-qq', '--color', 'never']' returned non-zero exit status 1.
PyMuPDF#1.23.17#next <class 'subprocess.CalledProcessError'> Command '['uv', 'add', 'PyMuPDF==1.23.17', '--exclude-newer', '2024-01-24T21:46:30', '--no-sync', '-qq', '--color', 'never']' returned non-zero exit status 1.


  2%|▋                                   | 980/48380 [02:33<17:24:24,  1.32s/it]

Scrapy#2.11.1#next <class 'subprocess.CalledProcessError'> Command '['uv', 'add', 'Scrapy==2.11.1', '--exclude-newer', '2024-05-15T12:00:40', '--no-sync', '-qq', '--color', 'never']' returned non-zero exit status 1.
Scrapy#2.11.2#current <class 'subprocess.CalledProcessError'> Command '['uv', 'add', 'Scrapy==2.11.2', '--exclude-newer', '2024-05-15T12:00:40', '--no-sync', '-qq', '--color', 'never']' returned non-zero exit status 1.


  2%|▊                                   | 1034/48380 [03:04<2:20:47,  5.60it/s]


KeyboardInterrupt: 